In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

<hr>

## **Load Data**

In [10]:
data = pd.read_csv('c:\\Users\\USER\\Desktop\\project\\End-to-End-main\\End-to-End-main\\Chrun Management\\dataset\\Telco-Customer-Churn.csv')

In order to reserve the original data, we make a copy of it.

In [11]:
df = data.copy()

In [36]:
df.head(3)

,customerID,gender,SeniorCitizen,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,No,1,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,34,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,2,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


<hr>

#### Check Variable Names, Order and Necessity

Column names provide a clear identification of the data contained within each column. By reviewing the column names, we can quickly understand the type of information stored in each column. This helps we make sense of the data and interpret it correctly.

In [12]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Dependents', 'tenure',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges',
       'TotalCharges', 'Churn'],
      dtype='object')

Now, we change the order of the columns:

In [13]:
col_order = ['customerID', 
             'Dependents', 'MultipleLines',  'OnlineSecurity',
             'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV','StreamingMovies','PaperlessBilling',
             'gender', 'InternetService', 'Contract',  'PaymentMethod',
             'SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 
             'Churn']

df = df[col_order]

#### Check Data Type

Checking the data types is an important step in data cleaning because it helps ensure the consistency, accuracy, and reliability of the data.

In [6]:
df.dtypes

customerID           object
Dependents           object
MultipleLines        object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
PaperlessBilling     object
gender               object
InternetService      object
Contract             object
PaymentMethod        object
SeniorCitizen         int64
tenure                int64
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

Based on data documentation, we must check whether the data type of variables is correct or not.
* `customerID`: String *(Lable)*
* `Dependents`: String *(Binary)*
* `MultipleLines`: String *(Binary)*
* `OnlineSecurity`: String *(Binary)*
* `OnlineBackup`: String *(Binary)*
* `DeviceProtection`: String *(Binary)*
* `TechSupport`: String *(Binary)*
* `StreamingTV`: String *(Binary)*
* `StreamingMovies`: String *(Binary)*
* `PaperlessBilling`: String *(Binary)*
* `gender`: String *(Categorical)*
* `InternetService`: String *(Categorical)*
* `Contract`: String *(Categorical)*
* `PaymentMethod`: String *(Categorical)*
* `SeniorCitizen`: Number *(Numerical)*
* `tenure`: Number *(Numerical)*
* `MonthlyCharges`: Number *(Numerical)*
* `TotalCharges`: Number *(Numerical)*
* `Churn`: String *(Binary)*

The result shows, there is something wrong about `TotalCharges`.

In [14]:
df['TotalCharges'].unique()

array(['29.85', '1889.5', '108.15', ..., '346.45', '306.6', '6844.5'],
      dtype=object)

it shows we must remove '' from numbers.

In [15]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

#### Handle Missing Values

Checking for missing values is an important step in data cleaning because missing values can have a significant impact on the quality and reliability of the data analysis. Missing values can lead to incomplete or inaccurate data, which can skew the analysis and produce misleading results. By identifying and handling missing values appropriately, we ensure the integrity and reliability of the data.

In [9]:
def nulls(df):
    null_values = pd.DataFrame(df.isnull().sum())
    null_values[1] = null_values[0] / len(df)
    null_values.columns = ['count','%pct']
    filtered_null = null_values[null_values['%pct'] > 0].sort_values(by='%pct', ascending=False)
    return filtered_null

nulls(df)

,count,%pct
TotalCharges,11,0.001562


>**Strategy One: Drop Column**

* The percentage of null value is more than 70%.

>**Strategy Two: Drop Rows:**

* The percentage of null value is less than 5%.

>**Strategy Three: Impute with Mode:**

* The percentage of null value is between 5% and 10%.
* The datatype is categorical or lable

>**Strategy Four: Impute with Mean or Median:**

* The percentage of null value is between 5% and 10%.
* The datatype is numerical

>**Strategy Five: Impute with KNN Method:**

* The percentage of null value is between 10% and 70%.
* The datatype is categorical lable
* They are not MNAR

>**Strategy Six: Impute with KNN Method:**

* The percentage of null value is between 10% and 70%.
* The datatype is Number
* They are not MNAR

We choose the startegy Two and remove the rows.

In [16]:
df = df.dropna(subset = ['TotalCharges'])

#### **1.4-Check Duplicate Rows**

Duplicate rows can compromise the integrity of the dataset. If we have multiple identical rows, it can lead to inaccurate statistical analysis, misleading results, and duplicate entries in downstream processes. By identifying and removing duplicate rows, we ensure that the data accurately represents the underlying information.

In [17]:
df.duplicated().sum()

0

As we can see, they are differenet policies based on the business line. So, they are not duplicated rows.

## Numbers

* Check Possible Range 
* Sanity Check

#### Check Possible Range

Checking the possible range is an important step in data cleaning because it helps identify and handle data values that fall outside the expected or valid range for a particular variable. By examining the range of values, we can identify outliers, data entry errors, or any other data points that are unlikely or impossible given the context of the data

In [18]:
num_var = [col for col in df.columns if df[col].dtypes != 'O']
num_var

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

In [19]:
for i in range(len(num_var)):
    per = round((len(df[df[num_var[i]] < 0]) / len(df)) * 100, 5)
    print('{} | min {} | max {} | percentage %{}'.format(num_var[i], df[num_var[i]].min(), df[num_var[i]].max(), per))

SeniorCitizen | min 0 | max 1 | percentage %0.0
tenure | min 1 | max 72 | percentage %0.0
MonthlyCharges | min 18.25 | max 118.75 | percentage %0.0
TotalCharges | min 18.8 | max 8684.8 | percentage %0.0


#### Sanity Check

Sanity checks can help ensure the consistency and integrity of the data. If two numerical variables are expected to have a certain relationship or dependency, a sanity check can help to understand. Among all numerical variables, first, we must realize there is a logical relationship between which variables. We should analye the relationship based on the data documentation.

* sanity 1 : `MonthlyCharges` <= `TotalCharges`

In [14]:
df['Sanity1'] = ['OK' if x <= y else 'NOT' for x, y in zip(df['MonthlyCharges'], df['TotalCharges'])]

In [20]:
df[df['Sanity1']=='NOT'].shape[0]

0

## String

* Trimming and Transformation 
* Check Possible Range 

#### Trimming and Transformation

Text data often contains unwanted leading or trailing spaces, which can affect data integrity and analysis. Trimming these spaces ensures consistency and accuracy in subsequent operations.

In [20]:
df[['LOB','Pol_type','Gender','Marital']] = df[['LOB','Pol_type','Gender','Marital']].applymap(str.strip).applymap(str.upper)

KeyError: "None of [Index(['LOB', 'Pol_type', 'Gender', 'Marital'], dtype='object')] are in the [columns]"

In [21]:
df.head(3)

,customerID,Dependents,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,gender,InternetService,Contract,PaymentMethod,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,No,No phone service,No,Yes,No,No,No,No,Yes,Female,DSL,Month-to-month,Electronic check,0,1,29.85,29.85,No
1,5575-GNVDE,No,No,Yes,No,Yes,No,No,No,No,Male,DSL,One year,Mailed check,0,34,56.95,1889.50,No
2,3668-QPYBK,No,No,Yes,Yes,No,No,No,No,Yes,Male,DSL,Month-to-month,Mailed check,0,2,53.85,108.15,Yes


#### Check Possible Range

By checking the possible range for categorical variables, we can identify any unexpected or erroneous values. This helps ensure the data's integrity and validity. It allows us to detect outliers, misspellings, or other inconsistencies in the categorical variable values.

In [22]:
def cat_checker(Var):
    unique_value = pd.Series(df[Var].unique()).sort_values().values
    return unique_value

In [23]:
cat_var = ['Dependents', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
           'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'gender',
           'InternetService', 'Contract', 'PaymentMethod']

for i in range(len(cat_var)):
    print('{}: {}'.format(cat_var[i], cat_checker(cat_var[i])))

Dependents: ['No' 'Yes']
MultipleLines: ['No' 'No phone service' 'Yes']
OnlineSecurity: ['No' 'No internet service' 'Yes']
OnlineBackup: ['No' 'No internet service' 'Yes']
DeviceProtection: ['No' 'No internet service' 'Yes']
TechSupport: ['No' 'No internet service' 'Yes']
StreamingTV: ['No' 'No internet service' 'Yes']
StreamingMovies: ['No' 'No internet service' 'Yes']
PaperlessBilling: ['No' 'Yes']
gender: ['Female' 'Male']
InternetService: ['DSL' 'Fiber optic' 'No']
Contract: ['Month-to-month' 'One year' 'Two year']
PaymentMethod: ['Bank transfer (automatic)' 'Credit card (automatic)' 'Electronic check'
 'Mailed check']


About `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`,  `TechSupport`, `StreamingTV`, `StreamingMovies`, No phone service means No.

In [24]:
change_lable_list = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                     'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',]

for i in range(len(change_lable_list)):
    df[change_lable_list[i]] = df[change_lable_list[i]].replace('No internet service', 'No')

Also, in `MultipleLines`, No phone service means No.

In [25]:
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')

In [26]:
for i in range(len(cat_var)):
    print('{}: {}'.format(cat_var[i], cat_checker(cat_var[i])))

Dependents: ['No' 'Yes']
MultipleLines: ['No' 'Yes']
OnlineSecurity: ['No' 'Yes']
OnlineBackup: ['No' 'Yes']
DeviceProtection: ['No' 'Yes']
TechSupport: ['No' 'Yes']
StreamingTV: ['No' 'Yes']
StreamingMovies: ['No' 'Yes']
PaperlessBilling: ['No' 'Yes']
gender: ['Female' 'Male']
InternetService: ['DSL' 'Fiber optic' 'No']
Contract: ['Month-to-month' 'One year' 'Two year']
PaymentMethod: ['Bank transfer (automatic)' 'Credit card (automatic)' 'Electronic check'
 'Mailed check']


## **Export the File**

First, we remove the Sanity column that we made it before, and then export the cleaned data.

In [32]:
df = df.drop(['Sanity1'], axis=1)
df.to_csv('dataset/Cleaned_df.csv', index=False)